In [ ]:
%%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install rouge_score
!pip install sentencepiece
!pip install transformers

In [ ]:
from datasets import load_dataset
import csv
import json
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

The data is from [Kaggle](https://www.kaggle.com/datasets/secareanualin/football-events)

In [ ]:
data_sets = pd.read_csv('/content/events.csv')
data_sets

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,NaN,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941004,z5L2OT5E/,z5L2OT5E123,123,92,Lucas Torreira (Sampdoria) wins a free kick in...,8,NaN,2,Sampdoria,Atalanta,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
941005,z5L2OT5E/,z5L2OT5E124,124,93,"Corner, Sampdoria. Conceded by Andrea Masiello.",2,NaN,2,Sampdoria,Atalanta,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
941006,z5L2OT5E/,z5L2OT5E125,125,93,Attempt missed. Fabio Quagliarella (Sampdoria)...,1,12.0,2,Sampdoria,Atalanta,...,NaN,NaN,8.0,2.0,0,9.0,1.0,1,3.0,0
941007,z5L2OT5E/,z5L2OT5E126,126,94,Alberto Grassi (Atalanta) wins a free kick on ...,8,NaN,1,Atalanta,Sampdoria,...,NaN,NaN,NaN,NaN,0,4.0,NaN,0,NaN,0


In [ ]:
data_sets.iloc[519]["text"]

'Attempt saved. GaA«l Danic (Valenciennes) header from the centre of the box is saved in the top left corner. Assisted by David Ducourtioux with a cross.'

In [ ]:
def check(str):
    string = "~!@#$^&*_+-/<>[]©ª¡«'"
    for i in string:
          if i in str:
            return True

In [ ]:
arr = []
for index, row in data_sets.iterrows():
    if check(row["text"]) == True:
        arr.append(index)

In [ ]:
data = data_sets.drop(arr)

In [ ]:
data

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,NaN,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941004,z5L2OT5E/,z5L2OT5E123,123,92,Lucas Torreira (Sampdoria) wins a free kick in...,8,NaN,2,Sampdoria,Atalanta,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
941005,z5L2OT5E/,z5L2OT5E124,124,93,"Corner, Sampdoria. Conceded by Andrea Masiello.",2,NaN,2,Sampdoria,Atalanta,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
941006,z5L2OT5E/,z5L2OT5E125,125,93,Attempt missed. Fabio Quagliarella (Sampdoria)...,1,12.0,2,Sampdoria,Atalanta,...,NaN,NaN,8.0,2.0,0,9.0,1.0,1,3.0,0
941007,z5L2OT5E/,z5L2OT5E126,126,94,Alberto Grassi (Atalanta) wins a free kick on ...,8,NaN,1,Atalanta,Sampdoria,...,NaN,NaN,NaN,NaN,0,4.0,NaN,0,NaN,0


In [ ]:
data.iloc[:50000].to_csv('football.csv', index = True)

In [ ]:
data = pd.read_csv("/content/football.csv")

In [ ]:
event_types = {0:'Announcement', 1:'Attempt', 2:'Corner', 3:'Foul', 4:'Yellow card', 5:'Second yellow card', 6:'Red card', 7:'Substitution', 8:'Free kick won', 9:'Offside', 10:'Hand ball', 11:'Penalty conceded'}
event_types2 = {12:'Key Pass', 13:'Failed through ball', 14:'Sending off', 15:'Own goal'}
sides = {1:'Home', 2:'Away'}
shot_places = {1:'Bit too high', 2:'Blocked', 3:'Bottom left corner', 4:'Bottom right corner', 5:'Centre of the goal', 6:'High and wide', 7:'Hits the bar', 8:'Misses to the left', 9:'Misses to the right', 10:'Too high', 11:'Top centre of the goal', 12:'Top left corner', 13:'Top right corner'}
shot_outcomes = {1:'On target', 2:'Off target', 3:'Blocked', 4:'Hit the bar'}
locations = {1:'Attacking half', 2:'Defensive half', 3:'Centre of the box', 4:'Left wing', 5:'Right wing', 6:'Difficult angle and long range', 7:'Difficult angle on the left', 8:'Difficult angle on the right', 9:'Left side of the box', 10:'Left side of the six yard box', 11:'Right side of the box', 12:'Right side of the six yard box', 13:'Very close range', 14:'Penalty spot', 15:'Outside the box', 16:'Long range', 17:'More than 35 yards', 18:'More than 40 yards', 19:'Not recorded'}
bodyparts = {1:'right foot', 2:'left foot', 3:'head'}
assist_methods = {0:'None', 1:'Pass', 2:'Cross', 3:'Headed pass', 4:'Through ball'}
situations = {1:'Open play', 2:'Set piece', 3:'Corner', 4:'Free kick'}

In [ ]:
data['event_type'].replace(event_types, inplace =True)
data["event_type2"].replace(event_types2, inplace=True)
data['side'].replace(sides, inplace =True)
data['shot_place'].replace(shot_places, inplace =True)
data['shot_outcome'].replace(shot_outcomes, inplace =True)
data['location'].replace(locations, inplace =True)
data['bodypart'].replace(bodyparts, inplace =True)
data['assist_method'].replace(assist_methods, inplace =True)
data['situation'].replace(situations, inplace =True)

In [ ]:
def build_source(index,row):
    str_ = '"'
    if pd.isnull(row["event_team"]) == False:
        str_ = str_ + "eventTeam[" + row["event_team"] + "], "
    if pd.isnull(row["player"]) == False:
        str_ = str_ + "player[" + row["player"] + "], "
    if pd.isnull(row["player2"]) == False :
        str_ = str_ + 'player2[' + row["player2"] + '], '
    if pd.isnull(row["event_type"]) == False:
        str_ = str_ + "eventType[" + row["event_type"] + "], "
    if pd.isnull(row["event_type2"]) == False:
        str_ = str_ + "eventType2[" + row["event_type2"] + "], "
    if pd.isnull(row["shot_place"]) == False:
        str_ = str_ + "shotPlace[" + row["shot_place"] + "], "
    if pd.isnull(row["shot_outcome"]) == False:
        str_ = str_ + "shotOutcome[" + row["shot_outcome"] + "], "
    if pd.isnull(row["location"]) == False:
        str_ = str_ + "location[" + row["location"] + "], "
    if pd.isnull(row["bodypart"]) == False:
        str_ = str_ + "bodyPart[" + row["bodypart"] + "], "
    if pd.isnull(row["assist_method"]) == False:
        str_ = str_ + "assistMethod[" + row["assist_method"] + "], "
    if pd.isnull(row["situation"]) == False:
        str_ = str_ + "situation[" + row["situation"] + "], "
    str_ = str_[:-2]+ '"'
    return str_

In [ ]:
source = []
for index, row in data.iterrows():
    source.append(build_source(index,row))

In [ ]:
data.insert(loc=len(data.columns), column='source', value=source)
data

,Unnamed: 0,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,...,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,source
0,0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,Attempt,Key Pass,Away,Hamburg SV,...,NaN,High and wide,Off target,0,Left side of the box,left foot,Pass,Open play,0,"""eventTeam[Hamburg SV], player[mladen petric],..."
1,1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",Corner,NaN,Home,Borussia Dortmund,...,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,"""eventTeam[Borussia Dortmund], player[dennis d..."
2,2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",Corner,NaN,Home,Borussia Dortmund,...,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,"""eventTeam[Borussia Dortmund], player[heiko we..."
3,3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,Foul,NaN,Home,Borussia Dortmund,...,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,"""eventTeam[Borussia Dortmund], player[sven ben..."
4,4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,Free kick won,NaN,Away,Hamburg SV,...,NaN,NaN,NaN,0,Defensive half,NaN,None,NaN,0,"""eventTeam[Hamburg SV], player[gokhan tore], e..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,61058,zsWw6qmi/,zsWw6qmi39,39,45,Foul by Gary Medel (FC Sevilla).,Foul,NaN,Away,Sevilla,...,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,"""eventTeam[Sevilla], player[gary medel], event..."
49996,61059,zsWw6qmi/,zsWw6qmi40,40,45,"Corner, Levante. Conceded by Coke.",Corner,NaN,Home,Levante,...,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,"""eventTeam[Levante], player[coke], player2[cok..."
49997,61060,zsWw6qmi/,zsWw6qmi41,41,45,Juanlu (Levante) is shown the yellow card.,Yellow card,NaN,Home,Levante,...,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,"""eventTeam[Levante], player[juanlu], eventType..."
49998,61061,zsWw6qmi/,zsWw6qmi42,42,45,Fernando Navarro (FC Sevilla) is shown the yel...,Yellow card,NaN,Away,Sevilla,...,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,"""eventTeam[Sevilla], player[fernando navarro],..."


In [ ]:
data.to_csv('football_events.csv', index = False)

In [ ]:
data = load_dataset("csv", data_dir="/content/", data_files="football_events.csv",split='train[:35000]')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7a10ac93241d9e2e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


In [ ]:
data = data.train_test_split(test_size=0.3)
data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type', 'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2', 'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal', 'location', 'bodypart', 'assist_method', 'situation', 'fast_break', 'source'],
        num_rows: 24500
    })
    test: Dataset({
        features: ['Unnamed: 0', 'id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type', 'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2', 'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal', 'location', 'bodypart', 'assist_method', 'situation', 'fast_break', 'source'],
        num_rows: 10500
    })
})

In [ ]:
def construct_input_for_batch(batch):
    """Construct input strings from a batch."""
    source = [' '.join(concepts) for concepts in batch["source"]]
    target = batch["text"]
    return source, target

In [ ]:
def batch_tokenize(batch, tokenizer, max_length=500):
    """Construct the batch (source, target) and run them through a tokenizer."""
    source, target = construct_input_for_batch(batch)
    res = {
        "input_ids": tokenizer(source)["input_ids"],
        "labels": tokenizer(
            target,
            padding="max_length",
            truncation=True,
            max_length=max_length
        )["input_ids"],
    }
    return res

In [ ]:
from transformers import T5Tokenizer

model_name = "t5-base"
max_length = 250

tokenizer = T5Tokenizer.from_pretrained(model_name)

train_data_tokenized = data['train'].map(
    lambda batch: batch_tokenize(batch, tokenizer, max_length=max_length),
    batched=True
)
valid_data_tokenized = data['test'].shard(num_shards=10, index=0).map(
    lambda batch: batch_tokenize(batch, tokenizer, max_length=max_length),
    batched=True
)

Map:   0%|          | 0/24500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

model_path = "google/t5-v1_1-base"
model = T5ForConditionalGeneration.from_pretrained(model_path)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [ ]:
from datasets import load_metric

rouge_scorer = load_metric("rouge")

def rouge_metric_builder(tokenizer):
    def compute_rouge_metrics(pred):
        """Utility to compute ROUGE during training."""
        labels_ids = pred.label_ids
        pred_ids = pred.predictions
        # All special tokens are removed.
        pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        labels_ids[labels_ids == -100] = tokenizer.pad_token_id
        label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
        # Compute the metric.
        rouge_results = rouge_scorer.compute(
            predictions=pred_str,
            references=label_str,
            rouge_types=["rouge2"],
            use_aggregator=True,
            use_stemmer=False,
        )
        return {
            "rouge2": round(rouge_results['rouge2'].mid.fmeasure, 4),
        }
    return compute_rouge_metrics

rouge_metric_fn = rouge_metric_builder(tokenizer)

In [ ]:
random_seed = 51
beam_size = 4

train_args = Seq2SeqTrainingArguments(
    output_dir="T5-E2E-FTB",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
    # optimization args, the trainer uses the Adam optimizer
    # and has a linear warmup for the learning rate
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-04,
    num_train_epochs=6,
    warmup_steps=1000,
    # misc args
    seed=random_seed,
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="rouge2",
    # generation
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=train_args,
    train_dataset=train_data_tokenized,
    eval_dataset=valid_data_tokenized,
    tokenizer=tokenizer,
    compute_metrics=rouge_metric_fn,
)

trainer._max_length = max_length
trainer._num_beams = beam_size

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: assist_method, player_in, player2, event_type, fast_break, bodypart, Unnamed: 0, event_type2, player, time, event_team, source, is_goal, opponent, sort_order, player_out, id_odsp, side, location, text, situation, shot_outcome, id_event, shot_place. If assist_method, player_in, player2, event_type, fast_break, bodypart, Unnamed: 0, event_type2, player, time, event_team, source, is_goal, opponent, sort_order, player_out, id_odsp, side, location, text, situation, shot_outcome, id_event, shot_place are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 24500
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9192
  Number of 

Epoch,Training Loss,Validation Loss,Rouge2
1,0.160400,0.105358,0.232900
2,0.134400,0.108657,0.066100
3,0.106300,0.089687,0.068600
4,0.087200,0.066602,0.079500
5,0.057300,0.033514,0.390100
6,0.043300,0.023777,0.498300


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: assist_method, player_in, player2, event_type, fast_break, bodypart, Unnamed: 0, event_type2, player, time, event_team, source, is_goal, opponent, sort_order, player_out, id_odsp, side, location, text, situation, shot_outcome, id_event, shot_place. If assist_method, player_in, player2, event_type, fast_break, bodypart, Unnamed: 0, event_type2, player, time, event_team, source, is_goal, opponent, sort_order, player_out, id_odsp, side, location, text, situation, shot_outcome, id_event, shot_place are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1050
  Batch size = 16
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder

TrainOutput(global_step=9192, training_loss=1.5535852444286031, metrics={'train_runtime': 7626.3006, 'train_samples_per_second': 19.275, 'train_steps_per_second': 1.205, 'total_flos': 8.665054349480755e+16, 'train_loss': 1.5535852444286031, 'epoch': 6.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/model_ftb.pth '/content/drive/MyDrive/Colab Notebooks'

In [ ]:
import torch
model = torch.load('/content/drive/MyDrive/Colab Notebooks/model_ftb.pth')

In [ ]:
def beam_generate_sentences(
    batch,
    model,
    tokenizer,
    num_beams=4,
    max_length=250,
    device='cuda:0'
):
    """Generate outputs from a model with beam search decoding."""
    # Create batch inputs.
    source, _ = construct_input_for_batch(batch)
    # Use the model's tokenizer to create the batch input_ids.
    batch_features = tokenizer(source, padding=True, return_tensors='pt')
    # Move all inputs to the device.
    batch_features = dict([(k, v.to(device)) for k, v in batch_features.items()])

    # Generate with beam search.
    generated_ids = model.generate(
        **batch_features,
        num_beams=num_beams,
        max_length=max_length,
    )

    # Use model tokenizer to decode to text.
    generated_sentences = [
        tokenizer.decode(gen_ids.tolist(), skip_special_tokens=True)
        for gen_ids in generated_ids
    ]
    return generated_sentences

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

from transformers import T5Tokenizer

model_name = "t5-base"
max_length = 250

tokenizer = T5Tokenizer.from_pretrained(model_name)

output = data['test'].shard(num_shards=10, index=0).map(
    lambda batch: {'generated': beam_generate_sentences(
        batch,
        model,
        tokenizer,
        num_beams=4,
        max_length=max_length,
        device=device)
    },
    batched=True,
    batch_size=16,
)

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_metric
meteor = load_metric("meteor")
meteor_results = meteor.compute(
    predictions=output["generated"],
    references=output["text"],
)

meteor_results

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


{'meteor': 0.7997210774646868}

In [ ]:
torch.save(model, 'model_ftb.pth')

In [ ]:
target_text = [o['text'] for o in output]
target_text[:90]

['Foul by Andreas Ottl (Hertha Berlin).',
 'Foul by Javi MartA\xadnez (Athletic de Bilbao).',
 'Attempt missed. Giorgio Chiellini (Juventus) header from the centre of the box misses to the left. Assisted by Emanuele Giaccherini with a cross following a corner.',
 'Foul by Martin Stranzl (Borussia Monchengladbach).',
 'Georges Mandjeck (Rennes) wins a free kick in the defensive half.',
 'Maximiliano Moralez (Atalanta) is shown the yellow card for a bad foul.',
 'Foul by Sylvain Marchal (St Etienne).',
 'Foul by Timm Klose (FC Nurnberg).',
 'Foul by Ivan Perisic (Borussia Dortmund).',
 'Corner,  Siena. Conceded by Davide Astori.',
 'Romain Danze (Rennes) is shown the yellow card for a bad foul.',
 'Attempt blocked. Mats Hummels (Borussia Dortmund) right footed shot from outside the box is blocked.',
 'Corner,  Montpellier. Conceded by Carlos Henrique.',
 'Foul by Ilan (Ajaccio).',
 'Attempt saved. Federico Peluso (Atalanta) header from the centre of the box is saved in the centre of the 

In [ ]:
generated_text = [o['generated'] for o in output]
generated_text[:90]

['Foul by Andreas Ottl (Hertha Berlin).',
 'Foul by Javi MartA\xadnez (Athletic de Bilbao).',
 'Attempt missed. Giorgio Chiellini (Juventus) header from the centre of the box misses to the left. Assisted by Emanuele CalaiA2 with a cross following a corner.',
 'Foul by Martin Stranzl (Borussia Monchengladbach).',
 'Geoffrey Dernis (Rennes) wins a free kick in the defensive half.',
 'Mathieu Bodmer (Athletic de Bilbao) wins a free kick in the defensive half.',
 'Foul by Sylvain Marchal (St Etienne).',
 'Foul by Tomas Pekhart (FC Nurnberg).',
 'Foul by Ivan Perisic (Borussia Dortmund).',
 'Corner, Siena. Conceded by Davide Astori.',
 'Rodney Strasser (Rennes) wins a free kick in the defensive half.',
 'Attempt blocked. Massimo Paci (Borussia Dortmund) right footed shot from outside the box is blocked.',
 'Corner, Milan. Conceded by Carlos Henrique.',
 'Foul by Ilan (Ajaccio).',
 'Attempt missed. Franck Ribery (Athletic de Bilbao) header from the centre of the box is close, but misses to t